In [1]:
# Imports needed for this ipython file

from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
import pickle
from TwitterAPI import TwitterAPI

%matplotlib inline

In [3]:
def get_twitter(config_file):
    """ 
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [4]:
def robust_request(twitter, resource, params, max_tries=2):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)    
        if request.status_code == 200:
            return request
        elif "Not authorized" in request.text:
            return -1
        elif "page does not exist" in request.text:
            return -2
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)

In [5]:
def get_followers(screen_name):
    """ Return a dictionary of the users that follows this person on Twitter.
    
    Args:
        screen_name: a string of a Twitter screen name
    Returns:
        A dictionary of dictionary, one per follower containing {index, user_info} pairs.
    Note: Many users follow more than 200 accounts; we will return those accounts too.
    """
    followers = {}
    cursor = -1
    key = 0
    rate_limit = 1
    while True:
        if rate_limit == 15:
            rate_limit = 1
            print >> sys.stderr, 'Avoided rate_limit error: \nsleeping for 15 minutes.'
            time.sleep(61 * 15)
        request = robust_request(twitter,'followers/list', 
                                 {'screen_name': screen_name, 'count':200, 'cursor': cursor})
        json_response = request.json()
        
        # Get 200 followers from json_response
        for follower in json_response['users']:
                followers[follower['screen_name']] = follower
                key = key + 1
                
        # If user has < 200 followers or to retrieve last 200 followers
        if json_response["next_cursor"] == 0:
            break
            
        # Update cursor to get next 200 followers
        cursor = json_response["next_cursor"]
        rate_limit = rate_limit + 1
        print "Collected %d followers" % key
    return followers

In [6]:
def get_followers_and_pickle_object(account_name):
    """ Calls get_followers() to collect all followers of a Twitter account
        and then saves it by pickling for future processing purposes.
    
    Args: A string representing the screen_name of a Twitter account,
          whose followers we need.
    Returns : A dictionary of { screen_name : user_info } pairs of followers,
              where user_info is a dictionary object containing user information
    """
    followers = get_followers(account_name)
    fileObject = open("raw_data/" + account_name,'wb')
    pickle.dump(followers,fileObject)
    fileObject.close()
    return followers

In [7]:
def get_followers_from_pickle(account_name):
    """ Returns all followers of a Twitter account by retrieving saved data,
        using pickle method.
    
    Args: A string representing the screen_name of a Twitter account,
          whose followers we need.
    Returns : A dictionary of { screen_name : user_info } pairs of followers,
              where user_info is a dictionary object containing user information
    """
    fileObject = open("raw_data/" + account_name,'r')
    followers = pickle.load(fileObject)
    fileObject.close()
    return followers

In [61]:
# Getting iit alumni association followers and pickling them
iit_alumni_followers = get_followers_and_pickle_object('iitalumniassn')

In [7]:
iit_alumni_followers = get_followers_from_pickle('iitalumniassn')
print "Found %d followers for iitalumniassn" %len(iit_alumni_followers.keys())

Found 1861 followers for iitalumniassn


In [6]:
# Getting iit career services followers and pickling them
iit_career_followers = get_followers_and_pickle_object('IITCareers')

In [8]:
iit_career_followers = get_followers_from_pickle('IITCareers')
print "Found %d followers for IITCareers" %len(iit_career_followers.keys())

Found 643 followers for IITCareers


In [ ]:
# Getting iit main university followers and pickling them
iit_univ_followers = get_followers_and_pickle_object('illinoistech')

In [9]:
iit_univ_followers = get_followers_from_pickle('illinoistech')
print "Found %d followers for IIT University" %len(iit_univ_followers.keys())

Found 5426 followers for IIT University


In [ ]:
# Getting uiuc main university followers and pickling them
uiuc_univ_followers = get_followers_and_pickle_object('Illinois_Alma')

In [16]:
uiuc_univ_followers = get_followers_from_pickle('Illinois_Alma')
print "Found %d followers for UIUC University" %len(uiuc_univ_followers.keys())

Found 40017 followers for UIUC University


In [ ]:
# Getting uiuc alumni followers and pickling them
uiuc_alumni_followers = get_followers_and_pickle_object('UIAA')

In [45]:
uiuc_alumni_followers = get_followers_from_pickle('UIAA')
print "Found %d followers for UIUC Alumni" %len(uiuc_alumni_followers.keys())

Found 5960 followers for UIUC Alumni


In [8]:
def get_all_screen_names(*args):
    """ Returns union of all followers of a particular university
    
    Args: Variables containing followers from different accounts in dictionary format.
    
    Returns : A set of intersection of keys from the dictionaries passed as input.
    """
    univ_followers = set()
    for followers in args:
        for user in followers.keys():
            univ_followers.add(user)
    return univ_followers

In [11]:
iit_followers = get_all_screen_names(iit_alumni_followers,iit_career_followers,iit_univ_followers)
print "Found %d unique accounts from %d accounts " %(len(iit_followers), 
            (len(iit_alumni_followers)+len(iit_career_followers)+len(iit_univ_followers)))

Found 6565 unique accounts from 7930 accounts 


In [13]:
# Pickling iit_followers for future calculations.
fileObject = open("processed_data/iit_followers",'wb')
pickle.dump(iit_followers,fileObject)
fileObject.close()

In [9]:
# Loading iit_followers from pickle.
fileObject = open("processed_data/iit_followers",'r')
iit_followers = pickle.load(fileObject)
fileObject.close()

In [10]:
def get_friends(screen_name):
    """ Return a dictionary of users that are friends of this person on Twitter.
    
    Args:
        screen_name: a string of a Twitter screen name
    Returns:
        A dictionary of {screen_name : description} per friend of the given account.
    """
    friends = {}
    cursor = -1
    key = 0
    while True:
        request = robust_request(twitter,'friends/list', 
                                 {'screen_name': screen_name, 'count':200, 'cursor': cursor})
        
        if request == None:
            return 0
        
        if request == -1 :    # Authentication error : Can't access follower data
            return -1
        elif request == -2:
            return -2
        
        json_response = request.json()
        
        # Get 200 friends from json_response
        for friend in json_response['users']:
                #friends.append(friend['screen_name']) -- to_remove
                friends[friend['screen_name']] = friend['description']
                key = key + 1
                
        # If user has < 200 friends or to retrieve last friends followers
        if json_response["next_cursor"] == 0:
            break
            
        # Update cursor to get next 200 friends
        cursor = json_response["next_cursor"]
        print "Collected %d friends for %s" % (key,screen_name)
    print "Returning --- %d friends for %s" % (key,screen_name)
    return friends

In [ ]:
friends_of_iit_followers = { x : get_friends(x) for x in iit_followers}

In [11]:
def get_friends_and_pickle_object(univ_followers, univ_name):
    """ Calls get_friends() to collect all friends of a Twitter account
        and then saves it by pickling.
    
    Args: A list of strings representing the screen_names of all followers of a university,
          A string representing the university's name.
          
    Returns : A dictionary of { String , [List] } pairs,
              where String is a university follower's screen_name
              and List is a list of screen_names of their friends
    """
    friends_of_univ_followers = {}
    for follower in univ_followers:
        friends = get_friends(follower)
        friends_of_univ_followers[follower] = friends
        fileObject = open("raw_data/friends_of_" + univ_name + "_followers",'wb')
        pickle.dump(friends_of_univ_followers,fileObject)
        fileObject.close()
    return friends_of_univ_followers

In [ ]:
get_friends_and_pickle_object(iit_followers,"iit")

In [22]:
for idx,follower in enumerate(iit_followers):
    if "DinkyShop" in follower:
        print idx
print len(iit_followers_2)

1548
5017


In [21]:
iit_followers_2 =  list(iit_followers)[1548:]

In [15]:
def get_friend_ids(screen_name):
    """ Return a dictionary of users that are friends of this person on Twitter.
    
    Args:
        screen_name: a string of a Twitter screen name
    Returns:
        A list of ids per friend of the given account.
    """
    friends = []
    cursor = -1
    key = 0
    while True:
        request = robust_request(twitter,'friends/ids', 
                                 {'screen_name': screen_name, 'count':5000, 'cursor': cursor})
        
        if request == None:
            return 0
        elif request == -1 :    # Authentication error : Can't access follower data
            return -1
        elif request == -2:   # Page does not exist error
            return -2
        
        json_response = request.json()
        
        # Get 5000 friends from json_response
        friends = json_response['ids']
        key = len(friends)
                
        # If user has < 200 friends or to retrieve last friends followers
        if json_response["next_cursor"] == 0:
            break
            
        # Update cursor to get next 200 friends
        cursor = json_response["next_cursor"]
        print "Collected %d friends for %s" % (key,screen_name)
    print "Returning --- %d friends for %s" % (key,screen_name)
    return friends

In [14]:
def get_friend_ids_and_pickle_object(univ_followers, univ_name):
    """ Calls get_friends() to collect all friends of a Twitter account
        and then saves it by pickling.
    
    Args: A list of strings representing the screen_names of all followers of a university,
          A string representing the university's name.
          
    Returns : A dictionary of { String , [List] } pairs,
              where String is a university follower's screen_name
              and List is a list of screen_names of their friends
    """
    friends_of_univ_followers = {}
    for follower in univ_followers:
        friends = get_friend_ids(follower)
        friends_of_univ_followers[follower] = friends
        fileObject = open("raw_data/friend_ids_of_" + univ_name + "_followers",'wb')
        pickle.dump(friends_of_univ_followers,fileObject)
        fileObject.close()
    return friends_of_univ_followers

In [16]:
get_friend_ids_and_pickle_object(iit_followers,"iit")

Returning --- 552 friends for mlacedog
Returning --- 16 friends for GetIITJEE
Returning --- 1268 friends for izenku
Returning --- 68 friends for VHSRedbirdsGO
Returning --- 48 friends for KaranPatel0008
Returning --- 858 friends for katknight
Returning --- 1724 friends for sparklebliss
Returning --- 11 friends for IITDBGroup
Returning --- 386 friends for StartupUChicago
Returning --- 430 friends for Lucia_RS

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 1681 friends for Circle_West
Returning --- 307 friends for neerajpkamath
Returning --- 2679 friends for ChicagoBlueSky
Returning --- 73 friends for kamlesh7252
Returning --- 11 friends for harsh_khadgawat
Returning --- 1536 friends for DigGraphSol
Returning --- 1 friends for nandkri
Returning --- 244 friends for azullah
Returning --- 196 friends for sonrisassmiles
Returning --- 110 friends for thineownthought
Returning --- 3 friends for EdgardoValle1
Returning --- 1149 friends for RajatMa45718129
Returning --- 1475 friends for EdUSACanada

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 336 friends for jeremygentile
Returning --- 1866 friends for sewing3
Returning --- 40 friends for DanielBekkouche
Returning --- 393 friends for RAJUPVKS
Returning --- 28 friends for JordanW36886577
Returning --- 135 friends for james_hilliard
Returning --- 648 friends for BridgeportBA
Returning --- 1971 friends for goodmanbitbad
Returning --- 209 friends for HarshManiar2
Returning --- 569 friends for Tripti_Says
Returning --- 165 friends for burakpilavci
Returning --- 44 friends for bahulkumar82
Returning --- 319 friends for nrk8286
Returning --- 103 friends for Miller_psych
Returning --- 96 friends for samar_reis

...

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 662 friends for homefieldhunter
Returning --- 200 friends for ChicagoEvent
Returning --- 200 friends for arvindbrfoc
Returning --- 1 friends for rajveerkumat
Returning --- 359 friends for aakashkumar3163
Returning --- 26 friends for NabenduPorey
Returning --- 272 friends for ankitgupta0910
Returning --- 1251 friends for GradPsychology
Returning --- 16 friends for unnat_singh
Returning --- 2000 friends for cm_ferrari3
Returning --- 24 friends for Deepakshukla005
Returning --- 93 friends for suncccc
Returning --- 128 friends for usabuildit
Returning --- 470 friends for industrial_tour
Returning --- 71 friends for RachelYanover

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 270 friends for RTGHoops
Returning --- 53 friends for N_K_V_K
Returning --- 23 friends for lcy88920
Returning --- 24 friends for surajitpal6
Returning --- 107 friends for SolarWerksllc
Returning --- 52 friends for kavidahanayaka
Returning --- 26 friends for chiragcec
Returning --- 102 friends for BibhashBarman2
Returning --- 2511 friends for DinkyShop
Returning --- 1985 friends for FundasMadeEasy

TwitterConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=5)